<a href="https://colab.research.google.com/github/seonae0223/machine_learning/blob/main/02_%EC%95%84%EC%9D%B4%ED%85%9C_%EA%B8%B0%EB%B0%98_%ED%98%91%EC%97%85_%ED%95%84%ED%84%B0%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
MOVIE_FILE_PATH = "/content/drive/MyDrive/Colab Notebooks/한경_MLOps/머신러닝/05. 추천시스템/data/ml-latest-small-20240809T045908Z-001/ml-latest-small/movies.csv"
RATING_FILE_PATH = "/content/drive/MyDrive/Colab Notebooks/한경_MLOps/머신러닝/05. 추천시스템/data/ml-latest-small-20240809T045908Z-001/ml-latest-small/ratings.csv"

In [ ]:
import pandas as pd

movies = pd.read_csv(MOVIE_FILE_PATH)
ratings = pd.read_csv(RATING_FILE_PATH)

In [ ]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [ ]:
# ratings와 movies를 합치기
rating_movies = pd.merge(ratings, movies, on='movieId')
rating_movies.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


# 사용자-영화 희소 행렬
Pivot을 통해 사용자-아이템 행렬로 변환(희소행렬)
- 희소행렬 : 대부분의 값이 비어 있는 행렬
- 모든 사용자가 모든 영화에 대해 평점을 부여하진 않는다.

In [ ]:
ratings_matrix = rating_movies.pivot_table("rating", index="userId", columns="title")
ratings_matrix.head()

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# NaN 값을 모두 0으로 변환 -> NaN이 있으면 유사도를 구할 수가 없다.
ratings_matrix = ratings_matrix.fillna(0)
ratings_matrix

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.5,3.5,0.0,0.0,0.0


# 영화-영화 유사도
- 사용자-영화 행렬을 전치하여 영화-사용자 행렬로 만든 다음 유사도를 구하기

In [ ]:
# 전치 행렬 만들기
ratings_matrix_T = ratings_matrix.T
ratings_matrix_T.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
'Hellboy': The Seeds of Creation (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Round Midnight (1986),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Salem's Lot (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Til There Was You (1997),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# 코사인 유사도 구하기
from sklearn.metrics.pairwise import cosine_similarity
item_sim = cosine_similarity(ratings_matrix_T, ratings_matrix_T)

# 유사도 행렬을 데이터 프레임으로 만들기
item_sim_df = pd.DataFrame(data=item_sim, index=ratings_matrix.columns, columns=ratings_matrix.columns)
item_sim_df

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.141653,0.000000,...,0.000000,0.342055,0.543305,0.707107,0.0,0.000000,0.139431,0.327327,0.000000,0.0
'Hellboy': The Seeds of Creation (2004),0.000000,1.000000,0.707107,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
'Round Midnight (1986),0.000000,0.707107,1.000000,0.000000,0.000000,0.0,0.176777,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
'Salem's Lot (2004),0.000000,0.000000,0.000000,1.000000,0.857493,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
'Til There Was You (1997),0.000000,0.000000,0.000000,0.857493,1.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
eXistenZ (1999),0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.211467,0.216295,0.097935,0.132489,...,0.000000,0.000000,0.000000,0.000000,0.0,1.000000,0.192259,0.000000,0.170341,0.0
xXx (2002),0.139431,0.000000,0.000000,0.000000,0.000000,0.0,0.089634,0.000000,0.276512,0.019862,...,0.069716,0.305535,0.173151,0.246482,0.0,0.192259,1.000000,0.270034,0.100396,0.0
xXx: State of the Union (2005),0.327327,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.156764,0.000000,...,0.000000,0.382543,0.177838,0.231455,0.0,0.000000,0.270034,1.000000,0.000000,0.0


In [ ]:
item_sim_df['[REC] (2007)'].sort_values(ascending=False)[:10]

,[REC] (2007)
title,
[REC] (2007),1.000000
Tremors 3: Back to Perfection (2001),0.623737
Dog Soldiers (2002),0.621035
Wolf Creek (2005),0.601937
Joy Ride (2001),0.589097
Scooby-Doo 2: Monsters Unleashed (2004),0.557882
[REC]³ 3 Génesis (2012),0.544207
Buried (2010),0.540871
Evil Dead (2013),0.537966


# 평점 예측
- 아이템 기반 유사도 협업 필터링으로 개인화된 영화 추천
$$
\hat{R}_{u, i}=\frac{\sum\big{(}S_{i,N} \cdot R_{u,N}\big{)}}{\sum\big{(}\big{|}S_{i,N}\big{|}\big{)}}
$$

In [ ]:
def predict_rating(ratings_arr, item_sim_arr) :
  # 내적하기(@)
  ratings_pred = ratings_arr @ item_sim_arr / np.array([np.abs(item_sim_arr).sum(axis=1)])

  return ratings_pred

In [ ]:
ratings_matrix.shape, item_sim.shape

((610, 9719), (9719, 9719))

In [ ]:
import numpy as np
ratings_pred = predict_rating(ratings_matrix.values, item_sim_df.values)
ratings_pred_df = pd.DataFrame(
    data=ratings_pred,
    index=ratings_matrix.index,
    columns=ratings_matrix.columns # 영화 컬럼
)

ratings_pred_df

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.070345,0.577855,0.321696,0.227055,0.206958,0.194615,0.249883,0.102542,0.157084,0.178197,...,0.113608,0.181738,0.133962,0.128574,0.006179,0.212070,0.192921,0.136024,0.292955,0.720347
2,0.018260,0.042744,0.018861,0.000000,0.000000,0.035995,0.013413,0.002314,0.032213,0.014863,...,0.015640,0.020855,0.020119,0.015745,0.049983,0.014876,0.021616,0.024528,0.017563,0.000000
3,0.011884,0.030279,0.064437,0.003762,0.003749,0.002722,0.014625,0.002085,0.005666,0.006272,...,0.006923,0.011665,0.011800,0.012225,0.000000,0.008194,0.007017,0.009229,0.010420,0.084501
4,0.049145,0.277628,0.160448,0.206892,0.309632,0.042337,0.130048,0.116442,0.099785,0.097432,...,0.051269,0.076051,0.055563,0.054137,0.008343,0.159242,0.100941,0.062253,0.146054,0.231187
5,0.007278,0.066951,0.041879,0.013880,0.024842,0.018240,0.026405,0.018673,0.021591,0.018841,...,0.009689,0.022246,0.013360,0.012378,0.000000,0.025839,0.023712,0.018012,0.028133,0.052315
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,0.246832,1.293006,0.752661,0.935924,1.032354,0.407038,0.516819,0.594590,0.489913,0.408661,...,0.227092,0.405114,0.291452,0.276143,0.055006,0.636525,0.510522,0.346652,0.550174,0.893777
607,0.052248,0.305255,0.180669,0.218834,0.179443,0.115288,0.165817,0.075548,0.104890,0.109500,...,0.059516,0.135763,0.093843,0.086643,0.003707,0.144222,0.134705,0.107674,0.156614,0.576894
608,0.324435,1.022541,0.598467,0.425468,0.349562,0.494081,0.529903,0.227746,0.480980,0.442384,...,0.276586,0.594918,0.457094,0.444436,0.038681,0.616733,0.717768,0.538586,0.527639,0.698871


# 예측 평가
가중치 평점 부여 후에 예측 성능 평가에 대한 MSE 구하기

In [ ]:
ratings_pred.nonzero() #0이 들어있지 않은 곳의 좌표값

'''
(array([  0,   0,   0, ..., 609, 609, 609]), -> 행번호
 array([   0,    1,    2, ..., 9716, 9717, 9718])) -> 열번호
 '''

(array([  0,   0,   0, ..., 609, 609, 609]),
 array([   0,    1,    2, ..., 9716, 9717, 9718]))

In [ ]:
# 실제 평점 데이터 중 원래 값이 들어있던 위치 구하기 -> target
# 예측 평점 데이터 중 실제 값이 들어있었던 위치의 값과 MSE를 계산
from sklearn.metrics import mean_squared_error


# 사용자가 평점을 부여한 영화에 대해서만 예측 성능 평가 MSE 구하기
def get_mse(pred, actual):
  # 실제 사용자가 평점을 부여한 위치의 데이터 (target)
  actual_y = actual[actual.nonzero()].flatten() # 0이 아닌 원래 있었던 데이터의 위치 구하기

  # 실제 사용자가 평점을 부여한 위치의 예측 데이터 (pred)
  predict_y = pred[actual.nonzero()].flatten() # 실제 사용자가 평점을 부여한 위치 값을 통한 예측 데이터

  return mean_squared_error(predict_y, actual_y)

get_mse(ratings_pred, ratings_matrix.values)

# 나중에 경사하강법 등과 같이 학습의 개념을 사용해야 성능이 더 좋아짐.

9.895354759094706

단순하게 유저-아이템 행렬과 아이템-아이템 유사도 행렬로 가중 평균 계산을 하면, **유사하지 않은 아이템들**도 평점 예측에 참여하기 때문에 예측 평점이 낮을 수 밖에 없다.

In [ ]:
# 추천 시스템을 만들고 최적화하기
# top-n 유사도를 가진 데이터들에 대해서만 예측 평점 계산 -> 유사도 값에서 유사도가 높은 값만 예측 평점 계산

def predict_rating_topsim(ratings_arr, item_sim_arr, n=20):
  # 사용자-아이템 평점 행렬 크기만큼 0으로 채운 예측 행렬 초기화
  pred = np.zeros(ratings_arr.shape)

  # 사용자-아이템 평점 행렬의 열 크기(영화의 개수)만큼 반복 수행
  for col in range(ratings_arr.shape[1]):

   # col 번째 영화와, 다른 모든 영화들 간의 유사도
   sim_items = item_sim_arr[:, col]
   top_n_items = [np.argsort(sim_items)[:-n-1:-1]] # 뒤쪽에서부터 20개의 높은 유사도가 뽑힘

   #개인화된 예측 평점 계산
   for row in range(ratings_arr.shape[0]):

    # item_sim_arr[col, :][top_n_items] : col 번째 영화와 가장 유사도가 높은 20개 영화의 유사도
    # ratings_arr[row, :][top_n_items].T : row 번째 사람이 부여한 유사도가 가장 높은 top_n 영화에 대한 점수
    pred[row, col] = item_sim_arr[col, :][top_n_items] @ ratings_arr[row, :][top_n_items].T
    pred[row, col] /=np.sum(np.abs(item_sim_arr[col, :][top_n_items]))

  return pred

In [ ]:
ratings_pred = predict_rating_topsim(ratings_matrix.values, item_sim_df.values)
get_mse(ratings_pred, ratings_matrix.values)

3.6949827608772314

In [ ]:
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index=ratings_matrix.index, columns=ratings_matrix.columns)
ratings_pred_matrix

# 보지 않을만한 영화는 아예 0점으로 처리됨.
# 유사하게 볼만한 영화들만 유사도가 표현
# 즉, 상위 20개의 유사도만 가지고 출력

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.220798,0.000000,0.000000,1.677291,0.284372
2,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.220798,0.000000,0.000000,0.194828,0.000000
5,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,0.0,0.149633,0.0,0.418273,0.16678,0.0,0.130033,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.344930,0.268465,0.000000,0.694944,0.189602
607,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.194948,0.000000,0.000000,0.000000,0.000000
608,0.0,0.000000,0.0,0.159451,0.00000,0.0,0.243703,0.0,0.000000,0.0,...,0.0,0.129289,0.000000,0.112856,0.0,1.587302,2.988072,0.175489,0.702430,0.000000


# 추천시스템 작동

In [ ]:
target_user_id = 78

In [ ]:
# target_user_id에 대한 모든 영화 정보
# 78번 id user가 본 영화에 대한 정보
user_rating_id = ratings_matrix.loc[target_user_id, :]

user_rating_id[user_rating_id > 0].sort_values(ascending=False)[:10]

,78
title,
Die Hard (1988),5.0
Airplane! (1980),5.0
"Terminator, The (1984)",5.0
Terminator 2: Judgment Day (1991),4.5
Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981),4.5
"Shawshank Redemption, The (1994)",4.5
Ghostbusters (a.k.a. Ghost Busters) (1984),4.5
"Matrix, The (1999)",4.5
Dodgeball: A True Underdog Story (2004),4.5


사용자가 보지 않은 영화 중에서 아이템 기반의 유사도 협업 필터링 추천

In [ ]:
def get_unseen_movies(ratings_matrix, userId):
  #userId로 입력 받은 사용자의 모든 영화 정보 추출.
  user_rating = ratings_matrix.loc[userId, :]

  # 이미 본 영화에 대한 인덱스 추출 (영화 제목)
  already_seen = user_rating[ user_rating > 0]. index.tolist()

  movie_list = ratings_matrix.columns.tolist()

  unseen_list = [ movie for movie in movie_list if movie not in already_seen]

  return unseen_list

In [ ]:
def recomm_movie_by_userId(pred_df, userId, unseen_list, top_n=10):
  recomm_movies = pred_df.loc[userId, unseen_list].sort_values(ascending=False)[:top_n]
  return recomm_movies

In [ ]:
# 사용자가 관람하지 않은 영화명 추출
unseen_list = get_unseen_movies(ratings_matrix, 90)

# 추천 목록 만들기
recomm_movies = recomm_movie_by_userId(ratings_pred_matrix, target_user_id, unseen_list, top_n=10)

# 평점 데이터를 DataFrame으로
recomm_movies_df = pd.DataFrame(data=recomm_movies.values, index=recomm_movies.index, columns=["pred_score"])
recomm_movies_df

,pred_score
title,
Terminator 2: Judgment Day (1991),2.927918
Die Hard (1988),2.633424
Star Wars: Episode VI - Return of the Jedi (1983),2.226222
Indiana Jones and the Last Crusade (1989),2.221552
Mission: Impossible (1996),2.136809
Apollo 13 (1995),2.103989
Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981),2.033715
Jurassic Park (1993),2.011825
"Terminator, The (1984)",2.009106
